## Initialisation

In [ ]:
include("model.jl")
import .SwarmModel as SM
using .SwarmModel

include("modelstats.jl")
import .ModelStats as MS
using .ModelStats

In [ ]:
using Plots, Measures

In [ ]:
import PyPlot; const plt = PyPlot
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["font.family"] = "serif"
rcParams["font.size"] = 8
rcParams["figure.dpi"] = 200
rcParams["lines.linewidth"] = 1
rcParams["axes.labelsize"] = 6
rcParams["axes.titlesize"] = 7
rcParams["legend.fontsize"] = 5
rcParams["xtick.labelsize"] = 6
rcParams["ytick.labelsize"] = 6
rcParams["text.usetex"] = false
rcParams["figure.figsize"] = (3.5, 3.5)

In [ ]:
function run_simulation_for_n_steps(b, parameters, n_steps=2000)
    for i in 1:n_steps
        compute_step(b; parameters...)
        apply_step(b)
    end
end

function show_swarm(config_file; system=nothing, overrides=nothing, goal=nothing, title="", 
                    xlims=nothing, ylims=nothing, step=nothing, xmargin=1, ymargin=1, 
                    xtick_rotation=0, compute=true, with_params=true, markersize=1, savedfigure=nothing)
    if system !== nothing
        b, parameters = system
    else
        b, parameters = load_swarm(config_file)
        parameters = overrides === nothing ? parameters : merge(parameters, overrides)
    end
    if goal !== nothing
        b[:,SM.GOAL_X:SM.GOAL_Y] .= goal
    end
    if compute
        compute_step(b; parameters...)
    end
    prm = findall(b[:,SM.PRM] .> 0.)
    _prm = setdiff(1:size(b)[1], prm)
    if with_params
        paramstring = SM.stringify_parameters(parameters)
    else
        paramstring = ""
    end
    sep = config_file == "" ? "" : " - "
    title = title * sep * splitext(basename(config_file))[1] * "\n" * paramstring
    if xlims == nothing
        xlims = minimum(b[:,SM.POS_X]) - xmargin, maximum(b[:,SM.POS_X]) + xmargin
    end
    if ylims == nothing
        ylims = minimum(b[:,SM.POS_Y]) - ymargin, maximum(b[:,SM.POS_Y]) + ymargin
    end
    fig, ax  = plt.subplots()
    ax.set(xlim=xlims, ylim=ylims)        # set the limits of the axes
    if step !== nothing
        plt.xticks([n for n in xlims[1]:step:xlims[2]], rotation=xtick_rotation)
        plt.yticks([n for n in ylims[1]:step:ylims[2]])
    end
    ax.set_xlabel("POS_X")
    ax.set_ylabel("POS_Y")
    ax.set_aspect("equal")
    ax.set_title(title)
    plt.grid(alpha=0.25)
    plt.subplots_adjust(left=0.2, top=0.85)
    ax.plot(b[_prm,SM.POS_X],b[_prm,SM.POS_Y], "ko", markersize=markersize)
    ax.plot(b[prm,SM.POS_X],b[prm,SM.POS_Y], "ro", markersize=markersize)
    if goal !== nothing
        ax.plot([goal[1]], [goal[2]], "b+", markersize=4)
    end
    if savedfigure !== nothing
        fig.savefig(savedfigure, bbox_inches="tight")
    end
    return fig, ax
end

function plot_mean_distances(config_file="config/base_400.json"; system=nothing, goal=nothing,
                             means=nothing, stds=nothing, n_steps=500, plots=[:ii, :pi, :pp],
                             k=[2,1,1,2], pre_p=false, boundary=50, with_stdev=false, 
                             overrides=nothing, failure=nothing, savedfigure=nothing, 
                             legend_loc="best", alt_file=nothing)
    if system !== nothing
        b, parameters = system
    else
        b, parameters = load_swarm(config_file)
        parameters = overrides === nothing ? parameters : merge(parameters, overrides)
    end
    if goal !== nothing
        b[:,SM.GOAL_X:SM.GOAL_Y] .= goal
    end
    if pre_p
        p = MS.agent_perimeter_status(b, parameters, n_steps=n_steps, boundary=boundary)
    else
        p = nothing
    end
    if means === nothing
        means, stds = knn_mean_distances(b, parameters, n_steps=n_steps, class_ids=plots,
                                         k=k, perimeter=p, failure=failure)
    end
    ptype = with_stdev ? "\\Psi_d" : "\\mu_d"
    if alt_file !== nothing
        file_basename = alt_file
    else
        file_basename = splitext(basename(config_file))[1]
    end
    title = "Distance metric by perimeter class - $(file_basename)\n $(SM.stringify_parameters(parameters))"
    labels = Dict(:ii => "\$$(ptype)(S_i, S_i, $(k[1]))\$",
                  :ip => "\$$(ptype)(S_i, S_p, $(k[2]))\$",
                  :pi => "\$$(ptype)(S_p, S_i, $(k[3]))\$",
                  :pp => "\$$(ptype)(S_p, S_p, $(k[4]))\$")
    # colors = Dict(:ii => "black", :ip => "green", :pi => "blue", :pp => "red")
    colors = Dict(:ii => "k-", :ip => "g-", :pi => "b-", :pp => "r-")
    facecolors = Dict(:ii => "black", :ip => "green", :pi => "blue", :pp => "red")
    # ribbons = [with_stdev ? stds[:, i] : nothing for i in 1:length(plots)]
    # plt = plot(plot_title=title, plot_titlefontsize=10, xlabel="Simulation step number",
    #            ylabel="\$$(ptype)\$")
    # for i in 1:length(plots)
    #     p = plots[i]
    #     plot!(plt, means[:, i]; ribbon=ribbons[i], color=colors[p], fillalpha=0.25,
    #     label=labels[p], plotargs...)
    # end
    fig, ax = plt.subplots(figsize=(3.5,2.5))
    ax.set_title(title)
    ax.set_xlabel("Simulation step number")
    ax.set_ylabel("\$$(ptype)\$")
    ymin = minimum(means) - maximum(stds)
    ymax = maximum(means) + maximum(stds)
    ax.set(xlim=(0,n_steps), ylim=(ymin, ymax))
    ax.grid(true)
    line = Array{Vector{plt.PyObject}, 1}(undef, length(plots))
    for i in 1:length(plots)
        p = plots[i]
        line[i] = ax.plot(means[:,i], colors[p], label=labels[p])
        # line[i].set_label(labels[p])
        if with_stdev
            ax.fill_between(1:n_steps, means[:,i] .+ stds[:,i], means[:,i] .- stds[:,i],
                            facecolor=facecolors[p], alpha=0.25)
        end
    end
    ax.legend(loc=legend_loc)
    plt.subplots_adjust(left=0.15, top=0.8, bottom=0.2)
    if savedfigure !== nothing
        fig.savefig(savedfigure, bbox_inches="tight")
    end
    # display(plt)
    return fig, ax, means, stds
end

## Reproduce graphs from paper

In [ ]:
show_swarm("../config/base_400.json", title="Initial swarm state", savedfigure="base_400_init_new.pdf")

In [ ]:
n_steps = 2000
config = "base_400"
config_file = "../config/$(config).json"
b, parameters = load_swarm(config_file)
run_simulation_for_n_steps(b, parameters, n_steps)
show_swarm(config_file, system=(b, parameters), title="Swarm state after $(n_steps) steps", savedfigure="$(config)_step_$(n_steps)_new.pdf")

In [ ]:
show_swarm("../config/base_hole_370.json", title="Initial swarm state", savedfigure="base_hole_370_init_new.pdf")

In [ ]:
n_steps = 2000
config = "base_hole_370"
config_file = "../config/$(config).json"
b, parameters = load_swarm(config_file)
run_simulation_for_n_steps(b, parameters, n_steps)
show_swarm(config_file, system=(b, parameters), title="Swarm state after $(n_steps) steps", savedfigure="$(config)_step_$(n_steps)_new.pdf")

In [ ]:
n_steps = 2000
config = "gap_fill_no_rgf_hole_370"
config_file = "../config/$(config).json"
b, parameters = load_swarm(config_file)
run_simulation_for_n_steps(b, parameters, n_steps)
show_swarm(config_file, system=(b, parameters), title="Swarm state after $(n_steps) steps", xlims=(-25, 20), ylims=(-25, 20), step=5, savedfigure="$(config)_step_$(n_steps)_new.pdf")

In [ ]:
n_steps = 2000
config = "gap_fill_with_rgf_hole_370"
config_file = "../config/$(config).json"
b, parameters = load_swarm(config_file)
run_simulation_for_n_steps(b, parameters, n_steps)
show_swarm(config_file, system=(b, parameters), title="Swarm state after $(n_steps) steps", savedfigure="$(config)_step_$(n_steps)_new.pdf")

In [ ]:
n_steps = 2000
config = "outer_400"
config_file = "../config/$(config).json"
b, parameters = load_swarm(config_file)
run_simulation_for_n_steps(b, parameters, n_steps)
show_swarm(config_file, system=(b, parameters), title="Swarm state after $(n_steps) steps", savedfigure="$(config)_step_$(n_steps)_new.pdf")

In [ ]:
n_steps = 2000
config = "inner_400"
config_file = "../config/$(config).json"
b, parameters = load_swarm(config_file)
run_simulation_for_n_steps(b, parameters, n_steps)
show_swarm(config_file, system=(b, parameters), title="Swarm state after $(n_steps) steps", savedfigure="$(config)_step_$(n_steps)_new.pdf")

In [ ]:
b, parameters = load_swarm("../config/inner_400.json")
for i in 1:2000
    compute_step(b; parameters...)
    apply_step(b)
end
prm = findall(b[:,SM.PRM] .> 0.)
scatter(b[:,SM.POS_X],b[:,SM.POS_Y]; legend=false, markersize=2, markercolor=:black, aspect_ratio=:equal)
scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=2, markercolor=:red, aspect_ratio=:equal)

In [ ]:
plot_mean_distances("../config/base_400.json", n_steps=2000)

In [ ]:
plot_mean_distances("../config/base_hole_370.json", n_steps=2000)

In [ ]:
plot_mean_distances("../config/gap_fill_no_rgf_hole_370.json", n_steps=2000)

In [ ]:
plot_mean_distances("../config/gap_fill_with_rgf_hole_370.json", n_steps=2000)

In [ ]:
plot_mean_distances("../config/inner_400.json", n_steps=2000, with_stdev=true)

In [ ]:
plot_mean_distances("../config/outer_400.json", n_steps=2000, with_stdev=true)

In [ ]:
plot_mean_distances("../config/outer_400.json", n_steps=2000, with_stdev=true, pre_p=true)

## Animation examples

In [ ]:
b, parameters = load_swarm("../config/gap_fill_with_rgf_hole_370.json")
@gif for i in 1:2000
    compute_step(b; parameters...)
    apply_step(b)
    prm = findall(b[:,SM.PRM] .> 0.)
    scatter(b[:,SM.POS_X],b[:,SM.POS_Y]; legend=false, markersize=3, markercolor=:black, aspect_ratio=:equal)
    scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=3, markercolor=:red, aspect_ratio=:equal, annotationfontsize=10, annotations=(20, 15, lpad(i, 5, "0")))    
end


In [ ]:
b, parameters = load_swarm("../config/gap_fill_with_rgf_hole_370.json")
parameters[:gain] = 0.0008
@gif for i in 1:2000
    compute_step(b; parameters...)
    apply_step(b)
    prm = findall(b[:,SM.PRM] .> 0.)
    scatter(b[:,SM.POS_X],b[:,SM.POS_Y]; legend=false, markersize=3, markercolor=:black, aspect_ratio=:equal)
    scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=3, markercolor=:red, aspect_ratio=:equal, annotationfontsize=10, annotations=(20, 15, lpad(i, 5, "0")))    
end

## Linear Control Examples

### Successes

In [ ]:
b, parameters = load_swarm("../config/gap_fill_with_rgf_hole_370.json")
parameters[:gain] = 0.0008
for i in 1:40000
    compute_step(b; parameters...)
    apply_step(b)
end
prm = findall(b[:,SM.PRM] .> 0.)
scatter(b[:,SM.POS_X],b[:,SM.POS_Y]; legend=false, markersize=2, markercolor=:black, aspect_ratio=:equal)
scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=2, markercolor=:red, aspect_ratio=:equal)

In [ ]:
plot_mean_distances("../config/gap_fill_with_rgf_hole_370.json", n_steps=40000, with_stdev=true, overrides=Dict(:gain => 0.0008))

In [ ]:
b, parameters = load_swarm("../config/inner_400.json")
parameters[:gain] = 0.0008
for i in 1:20000
    compute_step(b; parameters...)
    apply_step(b)
end
prm = findall(b[:,SM.PRM] .> 0.)
scatter(b[:,SM.POS_X],b[:,SM.POS_Y]; legend=false, markersize=2, markercolor=:black, aspect_ratio=:equal)
scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=2, markercolor=:red, aspect_ratio=:equal)

In [ ]:
plot_mean_distances("../config/inner_400.json", n_steps=20000, with_stdev=true, overrides=Dict(:gain => 0.0008))

The examples of `gap_fill_with_rgf_hole_370` and `inner_400` show that linear control can achieve good, stable swarm structures using a gain of 0.0008 with all other parameters as before. 

### A problem: `outer_400`

The next example, `outer_400`, looks less promising so far. The swarm structure graph below shows the swarm structure after 20000 steps, with gain of 0.0008 and other parameters left as before. The swarm becomes much more highly compressed and rather distorted.

In [ ]:
b, parameters = load_swarm("../config/outer_400.json")
parameters[:gain] = 0.0008
for i in 1:20000
    compute_step(b; parameters...)
    apply_step(b)
end
prm = findall(b[:,SM.PRM] .> 0.)
scatter(b[:,SM.POS_X],b[:,SM.POS_Y]; legend=false, markersize=2, markercolor=:black, aspect_ratio=:equal)
scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=2, markercolor=:red, aspect_ratio=:equal)

The knn-distance graph shows that the perimeter classes do settle down to something reasonably stable. Notice that pre-computation of the perimeter is required to get a sensible distance graph. The `boundary` parameter is used to select the cut-off point at which agents should be identified as perimeter agents. The boundary value used for this graph is 20, i.e. if an agent is identified as a perimeter agent in at least 20% (`boundary=20`) of the steps then it is classified as a perimeter agent. 

In [ ]:
plot_mean_distances("../config/outer_400.json", n_steps=20000, with_stdev=true, pre_p=true, boundary=20, saved_figure=true, overrides=Dict(:gain => 0.0008))

A boundary value of 20 seems like a low cut-off value. It is much lower than our usual value (`boundary=50`). However, the swarm structure graph below shows that in this case a boundary value of 20 is appropriate, as it identifies all, and only, those agents that would be identified as 'on the perimeter' by a human observer.

In [ ]:
b1, parameters1 = load_swarm("../config/outer_400.json")
parameters1[:gain] = 0.0008
p = MS.agent_perimeter_status(b1, parameters1, n_steps=20000, boundary=20)
prm = findall(p .== 2)
scatter(b[:,SM.POS_X],b[:,SM.POS_Y]; legend=false, markersize=2, markercolor=:black, aspect_ratio=:equal)
scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=2, markercolor=:red, aspect_ratio=:equal)

The distance graph below gives the results when a boundary value of 50 is used for this example. It can be seen that only the $(S_i, S_i)$ class of agents is present in the graph. This is because a boundary value of 50 identifies _no_ agents as perimeter agents, suggesting that the perimeter is highly unstable.

In [ ]:
plot_mean_distances("../config/outer_400.json", n_steps=20000, with_stdev=true, pre_p=true, boundary=50, overrides=Dict(:gain => 0.0008))

The final example shows an attempt to find some parameters to produce a well-structured swarm with a compact perimeter. This is the best I've been able to manage so far. It can be seen that the swarm is neither as well-structured, nor is its perimeter as compact, as in the normalised version. Maybe this is a prime target for a learning algorithm? Or perhaps you can find a better set of parameters?

In [ ]:
b, parameters = load_swarm("../config/outer_400.json")
parameters[:kd] = [0.,0.]
parameters[:gain] = 0.0008
parameters[:kc] = [0.15 5.0; 0.15 75.0]
parameters[:kr] = [20.0 2.0; 100.0 75.0]
parameters[:kg] = 350
for i in 1:60000
    compute_step(b; parameters...)
    apply_step(b)
end
prm = findall(b[:,SM.PRM] .> 0.)
scatter(b[:,SM.POS_X],b[:,SM.POS_Y]; legend=false, markersize=2, markercolor=:black, aspect_ratio=:equal)
scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=2, markercolor=:red, aspect_ratio=:equal, )
#annotationfontsize=10, annotations=(20, 10, lpad(i, 5, "0"))

In [ ]:
plot_mean_distances("../config/outer_400.json", n_steps=60000, with_stdev=true, pre_p=true, boundary=20, 
                    overrides=Dict(:kc => [0.15 5.0; 0.15 75.0], :kd => [0.,0.], :kr => [20.0 2.0; 100.0 75.0], :kg => 350, :gain => 0.0008))

In [ ]:
b, parameters = load_swarm("../config/outer_400.json")
parameters[:gain] = 0.0008
parameters[:kc] = [0.15 5.0; 0.15 75.0]
parameters[:kr] = [20.0 2.0; 100.0 75.0]
parameters[:kg] = 350
parameters[:kd] = [0., 0.]
@gif for i in 1:2000
    compute_step(b; parameters...)
    apply_step(b)
    prm = findall(b[:,SM.PRM] .> 0.)
    scatter(b[:,SM.POS_X],b[:,SM.POS_Y]; legend=false, markersize=2, markercolor=:black, aspect_ratio=:equal)
    scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=2, markercolor=:red, aspect_ratio=:equal, annotationfontsize=10, annotations=(16, 10, lpad(i, 5, "0")))
end


## Rotational vector

In [ ]:
n_steps = 10000
config = "low_density_400"
config_file = "../config/$(config).json"
b, parameters = load_swarm(config_file)
b[:,SM.GOAL_X:SM.GOAL_Y] .= [0.0 0.0]
run_simulation_for_n_steps(b, parameters, n_steps)
show_swarm(config_file, system=(b, parameters), goal=[0 0], title="Swarm state after $(n_steps) steps", xlims=(-150,150), ylims=(-150,150), step=25, 
           xmargin=5, ymargin=5, xtick_rotation=45, compute=false, with_params=false, savedfigure="$(config)_step_$(n_steps)_new.pdf")

In [ ]:
b, parameters = load_swarm("../config/low_density_400.json")
# parameters[:rb] = [16.0 16.0; 16.0 16.0]
# parameters[:cb] = 20.0
# parameters[:kc] = [0.25 0.075; 0.15 0.15]
# parameters[:kr] = [120.0 50.0; 120.0 50.0]
# parameters[:kd] = [0.2, 0.2]
# parameters[:ka] = [0.0, 15.0]
# parameters[:kg] = 150.0
# parameters[:rgf] = true
# parameters[:speed] = 0.005
b[:,SM.GOAL_X:SM.GOAL_Y] .= [0.0 0.0]
# parameters[:kr] = [150 50; 200 50]
# parameters[:ka] = [0, 150.0]
for i in 1:10000
    compute_step(b; parameters...)
    apply_step(b)
end
prm = findall(b[:,SM.PRM] .> 0.)
_prm = setdiff(1:size(b)[1], prm)
# scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; title="Swarm state after 10000 steps - low_density_400\n C:20.0 R:[[16.0, 16.0],[16.0, 16.0]]\n kc:[[0.25 0.075],[0.15,0.15]]\n kr:[[120.0, 50.0],[120.0,50.0]]\n kd:[0.2, 0.2] ka:[0.0, 15.0] kg:150.0 rgf:true", titlefontsize=8, labelfontsize=8, xtickfontrotation=45, legend=false, markersize=1, markercolor=:black, aspect_ratio=:equal, xlabel="POS_X", ylabel="POS_Y", xticks=[n for n in -150:25:150], yticks=[n for n in -150:25:150], framestyle=:box, size=(336,336))
scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; title="Swarm state after 10000 steps - low_density_400", titlefontsize=8, labelfontsize=8, xtickfontrotation=45, 
        legend=false, markersize=1, markercolor=:black, aspect_ratio=:equal, xlabel="POS_X", ylabel="POS_Y", xticks=[n for n in -150:25:150], yticks=[n for n in -150:25:150], 
        framestyle=:box, size=(336,336))
scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=1, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal)
plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)
savefig("low_density_400_step_10000.pdf")

In [ ]:
MS.centroid(b), MS.radius(b), MS.area(b), MS.density(b), 1 / MS.density(b)

In [ ]:
config = "low_density_360"
config_file = "../config/$(config).json"
failed = sort([324, 340, 231, 281, 132, 205, 13, 390, 122, 241, 59, 153, 126, 343, 211, 379, 105, 167, 273, 275, 193, 79, 121, 227, 302, 252, 83, 253, 367, 84, 11, 293, 98, 111, 68, 348, 41, 169, 352, 113])
b_ = [b[a,:] for a in 1:size(b)[1] if a ∉ failed]
b = collect(transpose(reshape(collect(Iterators.flatten(b_)),(20,360))))
show_swarm(config_file, system=(b, parameters), goal=[0 0], title="Swarm state after failure", xlims=(-150,150), ylims=(-150,150), step=25, 
           xmargin=5, ymargin=5, xtick_rotation=45, compute=false, with_params=false, savedfigure="$(config)_after_failure_new.pdf")

In [ ]:
failed = sort([324, 340, 231, 281, 132, 205, 13, 390, 122, 241, 59, 153, 126, 343, 211, 379, 105, 167, 273, 275, 193, 79, 121, 227, 302, 252, 83, 253, 367, 84, 11, 293, 98, 111, 68, 348, 41, 169, 352, 113])
#failed = sort([146, 340, 373, 21, 385, 231, 195, 281, 104, 399, 121, 70, 295, 122, 241, 105, 83, 334, 379, 200, 293, 302, 59, 167, 252, 98, 11, 253, 68, 238, 352, 348, 91, 41, 296, 156, 113, 367, 345])
# failed = sort([146, 340, 373, 21, 385, 231, 195, 281, 104, 399, 121, 70, 295, 122, 241, 105, 83, 334, 379, 200, 293, 302, 59, 167, 252, 98, 11, 253, 68, 238, 352, 348, 27, 41, 296, 156, 268, 132, 77, 345])
# failed = sort([393, 399, 241, 273, 330, 150, 275, 109, 46, 240, 274, 9, 171, 349, 347, 50, 136, 343, 227, 200, 379, 79, 314, 362, 107, 334, 135, 121, 27, 293, 302, 111, 62, 256, 105, 68, 83, 64, 352, 131])
b_ = [b[a,:] for a in 1:size(b)[1] if a ∉ failed]
b = collect(transpose(reshape(collect(Iterators.flatten(b_)),(20,360))))
prm = findall(b[:,SM.PRM] .> 0.)
_prm = setdiff(1:size(b)[1], prm)
scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; title="Swarm state after agent failure - low_density_360", titlefontsize=8, labelfontsize=8, xtickfontrotation=45, legend=false, markersize=1, markercolor=:black, aspect_ratio=:equal, xlabel="POS_X", ylabel="POS_Y", xticks=[n for n in -150:25:150], yticks=[n for n in -150:25:150], framestyle=:box, size=(336,336))
scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=1, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal)
plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)
# scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; title="Swarm state after agent failure - low_density_360\n C:20.0 R:[[16.0, 16.0],[16.0, 16.0]]\n kc:[[0.25 0.15],[0.15,0.15]]\n kr:[[120.0, 50.0],[120.0,50.0]]\n kd:[0.2, 0.2] ka:[0.0, 15.0] kg:150.0 rgf:true", titlefontsize=8, labelfontsize=8, xtickfontrotation=45, legend=false, markersize=1, markercolor=:black, aspect_ratio=:equal, xlabel="POS_X", ylabel="POS_Y", xticks=[n for n in -150:25:150], yticks=[n for n in -150:25:150], framestyle=:box, size=(336,336))
# scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=1, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal)
# plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)
# scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; title="Swarm state after agent failure - low_density_360\n C:12.0 R:[[8.0, 8.0],[8.0, 9.0]]\n kr:[[150.0, 50.0],[150.0,50.0]]\n kd:[0.15, 0.15] ka:[0.0, 10.0] kg:200.0 rgf:true", titlefontsize=8, labelfontsize=8, legend=false, markersize=1, markercolor=:black, aspect_ratio=:equal, xlabel="POS_X", ylabel="POS_Y", xticks=[n for n in -75:25:75], yticks=[n for n in -75:25:75], framestyle=:box, size=(336,336))
# scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=1, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal)
# plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)
savefig("low_density_360_after_failure.pdf")

In [ ]:
n_steps = 5000
config = "low_density_360"
config_file = "../config/$(config).json"
run_simulation_for_n_steps(b, parameters, n_steps)
show_swarm(config_file, system=(b, parameters), goal=[0 0], title="Swarm state after recovery", xlims=(-150,150), ylims=(-150,150), step=25, 
           xmargin=5, ymargin=5, xtick_rotation=45, compute=false, with_params=false, savedfigure="$(config)_step_$(n_steps)_new.pdf")

In [ ]:
for i in 1:5000
    compute_step(b; parameters...)
    apply_step(b)
end
prm = findall(b[:,SM.PRM] .> 0.)
_prm = setdiff(1:size(b)[1], prm)
scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; title="Swarm state after recovery - low_density_360", titlefontsize=8, labelfontsize=8, xtickfontrotation=45, legend=false, markersize=1, markercolor=:black, aspect_ratio=:equal, xlabel="POS_X", ylabel="POS_Y", xticks=[n for n in -150:25:150], yticks=[n for n in -150:25:150], framestyle=:box, size=(336,336))
scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=1, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal)
plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)
# scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; title="Swarm state after 10000 steps - low_density_360\n C:12.0 R:[[8.0, 8.0],[8.0, 9.0]]\n kr:[[150.0, 50.0],[150.0,50.0]]\n kd:[0.15, 0.15] ka:[0.0, 10.0] kg:200.0 rgf:true", titlefontsize=8, labelfontsize=8, legend=false, markersize=1, markercolor=:black, aspect_ratio=:equal, xlabel="POS_X", ylabel="POS_Y", xticks=[n for n in -75:25:75], yticks=[n for n in -75:25:75], framestyle=:box, size=(336,336))
# scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=1, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal)
# plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)
savefig("low_density_360_step_5000.pdf")

In [ ]:
MS.centroid(b), MS.radius(b), MS.area(b), MS.density(b)

In [ ]:
49638/54344

In [ ]:
plot_mean_distances("../config/low_density_400.json", goal=[0.0 0.0], n_steps=15000, failure=(10000, failed), with_stdev=true, 
    # plotargs=Dict(:legend=>:bottomright,:ylims=>[0,18], :framestyle=>:box, :top_margin=>20mm)
)

In [ ]:
b, parameters = load_swarm("../config/low_density_400.json")
b[:,SM.GOAL_X:SM.GOAL_Y] .= [0.0 0.0]
# parameters[:kc] = [0.25 0.075; 0.15 0.15]

# parameters[:rb] = [16.0 16.0; 16.0 16.0]
# parameters[:cb] = 20.0
# parameters[:kc] = [0.25 0.15; 0.15 0.15]
# parameters[:kr] = [120.0 50.0; 120.0 50.0]
# parameters[:kd] = [0.2, 0.2]
# parameters[:ka] = [0.0, 15.0]
# parameters[:kg] = 150.0
# parameters[:rgf] = true


# b[:,SM.GOAL_X:SM.GOAL_Y] .= [10.0 10.0]
# parameters[:kr] = [150 50; 200 50]
# parameters[:ka] = [0, 150.0]
n_steps = 10000
min_dist = Array{Float64, 1}(undef, n_steps)
for i in 1:n_steps
    _, _, mag, p = compute_step(b; parameters...)
    min_dist[i] = minimum(mag)
    apply_step(b)
end
plot(1:n_steps, min_dist)

In [ ]:
minimum(min_dist)

In [ ]:
minimum(min_dist)

In [ ]:
b, parameters = load_swarm("../config/low_density_400.json")
b[:,SM.GOAL_X:SM.GOAL_Y] .= [0.0 0.0]
# parameters[:kc] = [0.25 0.075; 0.15 0.15]

# parameters[:rb] = [16.0 16.0; 16.0 16.0]
# parameters[:cb] = 20.0
# parameters[:kc] = [0.25 0.15; 0.15 0.15]
# parameters[:kr] = [120.0 50.0; 120.0 50.0]
# parameters[:kd] = [0.2, 0.2]
# parameters[:ka] = [0.0, 15.0]
# parameters[:kg] = 150.0
# parameters[:rgf] = true

# parameters[:rb] = [16.0 16.0; 16.0 16.0]
# parameters[:cb] = 20.0
# parameters[:kc] = [0.15 0.015; 0.015 0.15]
# parameters[:kr] = [150.0 50.0; 150.0 50.0]
# parameters[:kd] = [0.2, 0.2]
# parameters[:ka] = [0.0, 15.0]
# parameters[:kg] = 150.0
# parameters[:rgf] = true
# parameters[:speed] = 0.005
# parameters[:rb] = [16.0 16.0; 16.0 16.0]
# parameters[:cb] = 20.0
# parameters[:kr] = [150.0 50.0; 100.0 50.0]
# parameters[:kd] = [0.2, 0.2]
# parameters[:ka] = [0.0, 15.0]
# parameters[:kg] = 150.0
# parameters[:rgf] = true
# parameters[:speed] = 0.005

# parameters[:rb] = [8.0 8.0; 8.0 8.0]
# parameters[:cb] = 10.0
# parameters[:kr] = [150.0 50.0; 150.0 50.0]
# parameters[:kd] = [0.2, 0.2]
# parameters[:ka] = [0.0, 15]
# parameters[:kg] = 150.0
# parameters[:rgf] = true


# parameters[:rb] = [8.0 8.0; 8.0 9.0]
# parameters[:cb] = 12.0
# parameters[:kr] = [150.0 50.0; 150.0 50.0]
# parameters[:kd] = [0.15, 0.15]
# parameters[:ka] = [0.0, 150.0]
# parameters[:kg] = 200.0
# parameters[:rgf] = true
# parameters[:gain] = 0.0008
gr()
@gif for i in 1:15001
    global b
    compute_step(b; parameters...)
    prm = findall(b[:,SM.PRM] .> 0.)
    _prm = setdiff(1:size(b)[1], prm)
    scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; title="Stable state of low density relative to initial state\n Rotating perimeter, self-healing", titlefontsize=10, labelfontsize=8, legend=false, markersize=3, markercolor=:black, aspect_ratio=:equal, size=(672, 672), xlabel="POS_X", ylabel="POS_Y", xlims=[-150,150], ylims=[-150,150], xticks=[n for n in -150:25:150], yticks=[n for n in -150:25:150])
    scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=3, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal, framestyle=:box, annotationfontsize=8, annotations=(125, 125, lpad(i-1, 5, "0")))
    plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)
    apply_step(b)
    if i == 10000
        failed = sort([324, 340, 231, 281, 132, 205, 13, 390, 122, 241, 59, 153, 126, 343, 211, 379, 105, 167, 273, 275, 193, 79, 121, 227, 302, 252, 83, 253, 367, 84, 11, 293, 98, 111, 68, 348, 41, 169, 352, 113])
        # failed = sort([146, 340, 373, 21, 385, 231, 195, 281, 104, 399, 121, 70, 295, 122, 241, 105, 83, 334, 379, 200, 293, 302, 59, 167, 252, 98, 11, 253, 68, 238, 352, 348, 27, 41, 296, 156, 268, 132, 77, 345])
        # failed = sort([146, 340, 373, 21, 385, 231, 195, 281, 104, 399, 121, 70, 295, 122, 241, 105, 83, 334, 379, 200, 293, 302, 59, 167, 252, 98, 11, 253, 68, 238, 352, 348, 91, 41, 296, 156, 113, 256, 35, 18])
        # failed = sort([393, 399, 241, 273, 330, 150, 275, 109, 46, 240, 274, 9, 171, 349, 347, 50, 136, 343, 227, 200, 379, 79, 314, 362, 107, 334, 135, 121, 27, 293, 302, 111, 62, 256, 105, 68, 83, 64, 352, 131])
        b_ = [b[a,:] for a in 1:size(b)[1] if a ∉ failed]
        b = collect(transpose(reshape(collect(Iterators.flatten(b_)),(SM.N_COLS, size(b)[1] - length(failed)))))
    end
end every 10

In [ ]:
MS.centroid(b), MS.radius(b), MS.area(b), MS.density(b)

In [ ]:
b = SM.mk_rand_swarm(10, grid=√10)
b[:,SM.GOAL_X:SM.GOAL_Y] .= [100.0 100.0]
parameters = deepcopy(SM.default_swarm_params)
parameters[:rb] = [96.0 96.0; 96.0 96.0]
parameters[:cb] = 144.0
parameters[:kr] = [150.0 50.0; 150.0 50.0]
parameters[:kd] = [0.15, 0.15 ]
parameters[:ka] = [0.0, 5.0]
parameters[:ra] = [0.0, π/2]
parameters[:kg] = 200.0
parameters[:rgf] = true
for i in 1:10000
    compute_step(b; parameters...)
    apply_step(b)
end
compute_step(b; parameters...)
prm = findall(b[:,SM.PRM] .> 0.)
_prm = setdiff(1:size(b)[1], prm)
scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; title="Rotation vectors", titlefontsize=10, labelfontsize=8, legend=false, xlabel="POS_X", ylabel="POS_Y", markersize=3, markercolor=:black, aspect_ratio=:equal, size=(336, 336), xlims=[50, 150], ylims=[50, 150])
scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=3, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal, framestyle=:box)
plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)
for i in 1:10
    # mdir = hypot(b[i,SM.DIR_X], b[i,SM.DIR_Y])
    # madv = hypot(b[i,SM.ADV_X], b[i,SM.ADV_Y])
    plot!([b[i,SM.POS_X], b[i,SM.POS_X] + b[i,SM.DIR_X]], [b[i,SM.POS_Y], b[i,SM.POS_Y] + b[i,SM.DIR_Y]], arrow=true, linecolor=:green)
    plot!([b[i,SM.POS_X], b[i,SM.POS_X] + b[i,SM.ADV_X]], [b[i,SM.POS_Y], b[i,SM.POS_Y] + b[i,SM.ADV_Y]], arrow=true, linecolor=:blue )
end

In [ ]:
b = SM.mk_rand_swarm(10, grid=√10)
b[:,SM.GOAL_X:SM.GOAL_Y] .= [100.0 100.0]
parameters = deepcopy(SM.default_swarm_params)
parameters[:rb] = [96.0 96.0; 96.0 96.0]
parameters[:cb] = 144.0
parameters[:kr] = [150.0 50.0; 150.0 50.0]
parameters[:kd] = [0.15, 0.15 ]
parameters[:ka] = [0.0, 5.0]
parameters[:ra] = [0.0, π/2]
parameters[:kg] = 200.0
parameters[:rgf] = true
n_steps = 10000
run_simulation_for_n_steps(b, parameters, n_steps)
config = "rotation_vectors"
config_file = "$(config).json"
fig, ax = show_swarm("", system=(b,parameters), title="Rotation vectors", goal=[100 100], xlims=(50, 150), ylims=(50,150), with_params=false, markersize=2)
for i in 1:10
    ax.arrow(b[i,SM.POS_X], b[i,SM.POS_Y], b[i,SM.DIR_X], b[i,SM.DIR_Y], width=0.0005, head_width=0.2, head_length=0.1, color="green")
    ax.arrow(b[i,SM.POS_X], b[i,SM.POS_Y], b[i,SM.ADV_X], b[i,SM.ADV_Y], width=0.0005, head_width=0.2, head_length=0.1, color="blue")
end
plt.savefig("rotation_vectors_new.pdf", bbox_inches="tight")

In [ ]:
MS.centroid(b), MS.radius(b), MS.area(b), size(b)[1] / MS.area(b)

In [ ]:
for i in 1:10
    # mdir = hypot(b[i,SM.DIR_X], b[i,SM.DIR_Y])
    # madv = hypot(b[i,SM.ADV_X], b[i,SM.ADV_Y])
    plot!([b[i,SM.POS_X], b[i,SM.POS_X] + b[i,SM.DIR_X]], [b[i,SM.POS_Y], b[i,SM.POS_Y] + b[i,SM.DIR_Y]], arrow=true, linecolor=:green)
    plot!([b[i,SM.POS_X], b[i,SM.POS_X] + b[i,SM.ADV_X]], [b[i,SM.POS_Y], b[i,SM.POS_Y] + b[i,SM.ADV_Y]], arrow=true, linecolor=:blue )
end

In [ ]:
savefig("rotation_vectors.pdf")

In [ ]:
gr()
b = SM.mk_rand_swarm(10, grid=√10)
b[:,SM.GOAL_X:SM.GOAL_Y] .= [100.0 100.0]
parameters = deepcopy(SM.default_swarm_params)
parameters[:rb] = [96.0 96.0; 96.0 96.0]
parameters[:cb] = 144.0
parameters[:kr] = [150.0 50.0; 150.0 50.0]
parameters[:kd] = [0.15, 0.15 ]
parameters[:ka] = [0.0, 5.0]
parameters[:ra] = [0.0, π/2]
parameters[:kg] = 200.0
parameters[:rgf] = true
# parameters[:rb] = [96.0 96.0; 96.0 108.0]
# parameters[:cb] = 144.0
# parameters[:kr] = [150.0 50.0; 150.0 75.0]
# parameters[:kd] = [0.0, 1.25]
# parameters[:ka] = [0.0, 5.0]
# parameters[:ra] = [0.0, π/2]
# parameters[:kg] = 200.0
# parameters[:rgf] = true

sb = deepcopy(b)
sp = deepcopy(parameters)

@gif for i in 1:10001
    global b
    compute_step(b; parameters...)
    prm = findall(b[:,SM.PRM] .> 0.)
    _prm = setdiff(1:size(b)[1], prm)
    scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; titlefontsize=10, labelfontsize=8, title="Goal seeking, rotating perimeter (border patrol)", legend=false, markersize=3, markercolor=:black, aspect_ratio=:equal, framestyle=:box, annotationfontsize=8, annotations=(150,150, lpad(i-1, 5, "0")), xlabel="POS_X", ylabel="POS_Y", xlims=[-50,160], ylims=[-50,160])
    scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=3, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal)
    plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)
    apply_step(b)
end every 5

In [ ]:
MS.centroid(b), MS.radius(b), MS.area(b), size(b)[1] / MS.area(b)

In [ ]:
gr()
# pyplot()
system = (deepcopy(sb), deepcopy(sp))
MS.plot_mean_distances(system=system, goal=[100.0 100.0], n_steps=1000, with_stdev=true, plotargs=Dict(:legend=>:bottomright,:ylims=>[0,30], :framestyle=>:box, :top_margin=>20mm, 
                                                                                                       :xticks=>[n for n in 0:100:1000], :yticks=>[n for n in 0:5:30]),
                       alt_file="border_patrol_10")
savefig("border_patrol_ds_1000.pdf")

In [ ]:
gr()
b = SM.mk_rand_swarm(20, grid=√20)
b[:,SM.GOAL_X:SM.GOAL_Y] .= [100.0 100.0]
parameters = deepcopy(SM.default_swarm_params)
# parameters[:rb] = [96.0 96.0; 96.0 108.0]
# parameters[:cb] = 144.0
parameters[:rb] = [48.0 48.0; 48.0 54.0]
parameters[:cb] = 54.0
# parameters[:kc] = [0.0 1.0; 1.0 1.0]
# parameters[:kr] = [150.0 50.0; 50.0 50.0]
parameters[:kc] = [0.0 1.0; 1.0 1.0]
parameters[:kr] = [150.0 5.0; 5.0 100.0]
# parameters[:kd] = [0.0, 0.5]
parameters[:kd] = [0.0, 1.0]
parameters[:ka] = [0.0, 10.0]
parameters[:ra] = [0.0, π/2]
# parameters[:kg] = 300.0
parameters[:kg] = 400.0
parameters[:rgf] = true

sb = deepcopy(b)
sp = deepcopy(parameters)

@gif for i in 1:20001
    global b
    compute_step(b; parameters...)
    prm = findall(b[:,SM.PRM] .> 0.)
    _prm = setdiff(1:size(b)[1], prm)
    scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; titlefontsize=10, labelfontsize=8, title="Goal seeking, rotating perimeter (border patrol)", legend=false, markersize=3, markercolor=:black, aspect_ratio=:equal, framestyle=:box, annotationfontsize=8, annotations=(175,175, lpad(i-1, 5, "0")), xlabel="POS_X", ylabel="POS_Y", xlims=[-50,200], ylims=[-50,200])
    scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=3, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal)
    plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)
    apply_step(b)
end every 5

In [ ]:
gr()
b = SM.mk_rand_swarm(60, grid=√60)
b[:,SM.GOAL_X:SM.GOAL_Y] .= [100.0 100.0]
parameters = deepcopy(SM.default_swarm_params)
parameters[:rb] = [96.0 96.0; 96.0 108.0]
parameters[:cb] = 144.0
# parameters[:kc] = [0.0 1.5; 1.0 0.5]
# parameters[:kr] = [150.0 25.0; 25.0 25.0]
parameters[:kc] = [0.0 5.0; 2.0 0.1]
parameters[:kr] = [150.0 0.0; 0.0 150.0]
parameters[:kd] = [0.35, 0.35]
parameters[:ka] = [0.0, 7.0]
parameters[:ra] = [0.0, π/2]
parameters[:kg] = 300.0
# parameters[:kg] = 200.0
parameters[:rgf] = true

sb = deepcopy(b)
sp = deepcopy(parameters)

@gif for i in 1:20001
    global b
    compute_step(b; parameters...)
    prm = findall(b[:,SM.PRM] .> 0.)
    _prm = setdiff(1:size(b)[1], prm)
    scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; titlefontsize=10, labelfontsize=8, title="Goal seeking, rotating perimeter (border patrol)", legend=false, markersize=3, markercolor=:black, aspect_ratio=:equal, framestyle=:box, annotationfontsize=8, annotations=(175,175, lpad(i-1, 5, "0")), xlabel="POS_X", ylabel="POS_Y", xlims=[-100,300], ylims=[-100,300])
    scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=3, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal)
    plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)
    apply_step(b)
end every 5

In [ ]:
gr()
b = SM.mk_rand_swarm(20, grid=√20)
b[:,SM.GOAL_X:SM.GOAL_Y] .= [100.0 100.0]
parameters = deepcopy(SM.default_swarm_params)
parameters[:rb] = [96.0 96.0; 96.0 108.0]
parameters[:cb] = 144.0
parameters[:kc] = [0.0 1.0; 1.0 1.0]
parameters[:kr] = [150.0 50.0; 50.0 50.0]
parameters[:kd] = [0.0, 0.3]
parameters[:ka] = [0.0, 2.5]
parameters[:ra] = [0.0, π/2]
parameters[:kg] = 200.0
parameters[:rgf] = true

compute_step(b; parameters...)
prm = findall(b[:,SM.PRM] .> 0.)
_prm = setdiff(1:size(b)[1], prm)
scatter(b[_prm,SM.POS_X],b[_prm,SM.POS_Y]; titlefontsize=10, labelfontsize=8, title="Goal seeking, rotating perimeter (border patrol)", legend=false, markersize=3, markercolor=:black, aspect_ratio=:equal, framestyle=:box, xlabel="POS_X", ylabel="POS_Y")
scatter!(b[prm,SM.POS_X],b[prm,SM.POS_Y]; legend=false, markersize=3, markercolor=:red, markerstrokecolor=:red, aspect_ratio=:equal)
# plot!([b[:,SM.GOAL_X][1]], [b[:,SM.GOAL_Y][1]], markershape=:cross, markersize=4)


In [ ]:
fig, ax = plt.subplots()
line = Array{Vector{plt.PyObject}, 1}(undef, 2)
m = rand(10,10)
for i in 1:2
    line[i] = ax.plot(m[:,i])
end

In [ ]:
typeof(line[1])